<p style="font-size:35px; text-align:center; font-weight:bold">CMAAS run SOM clustering method</p>
<p style="font-size:17px; text-align:left">Ina Storch 06-11-2023 </p>
<p style="font-size:17px; text-align:left">Note: This notebook is designed to run SOM using preprocessed data gained from the datacube from: Lawley et al., 2021. </p>
<p style="font-size:17px; text-align:left">Reference: Lawley, C.J.M., McCafferty, A.E., Graham, G.E., Gadd, M.G., Huston, D.L., Kelley, K.D., Paradis, S., Peter, J.M., and Czarnota, K., 2021. Datasets to support prospectivity modelling for sediment-hosted Zn-Pb mineral systems; Geological Survey of Canada, Open File 8836, 1 .zip file. https://doi.org/10.4095/329203</p>

<p style="font-size:19px; text-align:left; font-weight:bold">1) Import libraries, define SOM arguments</p>

In [1]:
from src.nextsomcore.nextsomcore import NxtSomCore
import pickle

import configs.argsSOM

args = configs.argsSOM.Args()

<p style="font-size:19px; text-align:left; font-weight:bold">2) Specify parameter for SOM. 

Input data can eighter be in LRN file format (a table with X, Y, Z and one line with the values for each data layer) or TIF file format. Choose one.

Create a "data" folder within the "methods/som/" folder. This "data" folder should contain a folder for "input" and "output" data, each. To be able to run this jupyter notebook, copy your input data into the folder "methods/som/data/input". 

In case of GeoTIF input files, you can create a subfolder and collect all TIF files that you want to use for SOM. In the following code cell you can provide a file path and a pattern (like "*.tif") for files within that specific folder that you want to use for SOM.

If input file format is LRN, skip the next code cell and provide the file name (including file path) with args.input_file 

In [2]:
# #------------- 
# #- Data Input .tiff files:
# #------------- 
# #- If input data is geotiff: list geotiff files: ["name1.tiff","name2.tiff"]
#in_folder = "data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/"
#input_list_text=[file_path + file for file in 
#                    ["Geology_BlackShale_Proximity.tif",
#                    "Terrane_Proximity.tif"]]
#args.input_file= ",".join(input_list_text)

file_path = "data/input/MVT_PREFERRED_REPROJECTED/"
file_patterns = ["NUMERICAL_MINMAX/*.tif", "CATEGORICAL/*_Igneous_Intrusive_Felsic.tif"]
#file_patterns = ["NUMERICAL_MINMAX/Magnetic_LongWavelength_HGM*.tif", "CATEGORICAL/*_Igneous_Intrusive_Felsic.tif"]
#file_patterns = ["NUMERICAL_MINMAX/Mag*.tif"]

args.input_file = args.create_list_from_pattern(file_path, file_patterns)
args.geotiff_input=args.input_file      # geotiff_input files, separated by komma

#------------- 
#- Or: Data Input .lrn file:
#------------- 
#args.input_file="/methods/methods/som/data/input/SOM_grav_mag.lrn"

#-------------
#- Data Output
#-------------

#args.output_folder="data/output"         # Folder to save som dictionary and cluster dictionary
args.output_folder="/methods/methods/som/data/output"

args.output_file_somspace= args.output_folder+"/result_som.txt"   # DO NOT CHANGE! Text file that will contain calculated values: som_x som_y b_data1 b_data2 b_dataN umatrix cluster in geospace.
        

#-------------
#- Parameter
#-------------

args.som_x=30                # X dimension of generated SOM
args.som_y=30                # Y dimension of generated SOM
args.epochs=10               # Number of epochs to run

# Base parameters required for som calculation. 
# Additional optional parameters below:
args.outgeofile= args.output_folder+"/result_geo.txt"             # DO NOT CHANGE!
args.output_file_geospace=args.outgeofile   # Text file that will contain calculated values: {X Y Z} data1 data2 dataN som_x som_y cluster b_data1 b_data2 b_dataN in geospace.

args.kmeans="true"          # Run k-means clustering (true, false)
args.kmeans_init=5           # Number of initializations
args.kmeans_min=3            # Minimum number of k-mean clusters
args.kmeans_max=25           # Maximum number of k-mean clusters

args.neighborhood='gaussian'     # Shape of the neighborhood function. gaussian or bubble
args.std_coeff=0.5               # Coefficient in the Gaussian neighborhood function
args.maptype='toroid'            # Type of SOM (sheet, toroid)
args.initialcodebook=None        # File path of initial codebook, 2D numpy.array of float32.
args.radius0=0                   # Initial size of the neighborhood
args.radiusN=1                   # Final size of the neighborhood
args.radiuscooling='linear'      # Function that defines the decrease in the neighborhood size as the training proceeds (linear, exponential)
args.scalecooling='linear'       # Function that defines the decrease in the learning scale as the training proceeds (linear, exponential)
args.scale0=0.1                  # Initial learning rate
args.scaleN=0.01                 # Final learning rate
args.initialization='random'     # Type of SOM initialization (random, pca)
args.gridtype='rectangular'      # Type of SOM grid (hexagonal, rectangular)
#args.xmlfile="none"              # SOM inputs as an xml file

args.normalized="true"      # Whether the data has been normalized or not (true, false)
args.minN=0                  # Minimum value for normalization
args.maxN=1                  # Maximum value for normalization
args.label=None              # Whether data contains label column, true or false


Number of files added: 20
Files:
data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Geology_BlackShale_Proximity.tif
data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Geology_Fault_Proximity.tif
data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Geology_Paleolatitude_Period_Maximum.tif
data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Geology_Paleolatitude_Period_Minimum.tif
data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Geology_PassiveMargin_Proximity.tif
data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Gravity_Bouguer.tif
data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Gravity_Bouguer_HGM.tif
data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Gravity_Bouguer_HGM_Worms_Proximity.tif
data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Gravity_Bouguer_UpCont30km_HGM.tif
data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Gravity_Bouguer_UpCont30km_HGM_Worms_Proximity.tif
...
data/input/MVT_PREFERRED_REPROJECTED/CATEGORICAL/Geology_Lithology_Minorit

In [3]:
print(args.input_file)
print(args.geotiff_input)

data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Geology_BlackShale_Proximity.tif,data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Geology_Fault_Proximity.tif,data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Geology_Paleolatitude_Period_Maximum.tif,data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Geology_Paleolatitude_Period_Minimum.tif,data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Geology_PassiveMargin_Proximity.tif,data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Gravity_Bouguer.tif,data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Gravity_Bouguer_HGM.tif,data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Gravity_Bouguer_HGM_Worms_Proximity.tif,data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Gravity_Bouguer_UpCont30km_HGM.tif,data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Gravity_Bouguer_UpCont30km_HGM_Worms_Proximity.tif,data/input/MVT_PREFERRED_REPROJECTED/NUMERICAL_MINMAX/Gravity_GOCE_ShapeIndex.tif,data/input/MVT_PREFERRED_REPR

<p style="font-size:19px; text-align:left; font-weight:bold">3) Run SOM 

Before running SOM - clean up existing files and move them to a subfolder.

In [4]:
import shutil
import os
import glob

file_path = args.output_folder
file_patterns = ["*som.*","*geo.*", "RunStats.txt","cluster.dictionary"]
destination_path = file_path+"/old_results/"

# Create the destination folder if it doesn't exist
if not os.path.exists(destination_path):
    os.makedirs(destination_path)

for file_pattern in file_patterns:
    # Use glob to get all files with the specified pattern
    matching_files = glob.glob(os.path.join(file_path, file_pattern))

    # Move each matching file to the destination folder and overwrite existing files if necessary
    for source_file in matching_files:
        file_name = os.path.basename(source_file)
        destination_file_path = os.path.join(destination_path, file_name)

        # If the file already exists in the destination folder, delete it first
        if os.path.exists(destination_file_path):
            os.remove(destination_file_path)

        # Move the file to the destination folder
        shutil.move(source_file, destination_file_path)

Run SOM with parameters specified above and save the results. Uses NxtSomCore package to do the actual work. 

In [5]:
import src.do_nextsomcore_save_results as dnsr

dnsr.run_SOM(args)

     10

Time for epoch 1: 1.667 Time for epoch 2: 1.654 Time for epoch 3: 1.476 Time for epoch 4: 1.515 Time for epoch 5: 1.426 Time for epoch 6: 1.513 Time for epoch 7: 1.422 Time for epoch 8: 1.413 Time for epoch 9: 1.25 Time for epoch 10: 1.323 

    100% [===================================================]


KeyboardInterrupt: 

In [ ]:
import src.plot_som_results as plot

# Load cluster dictionary
loaded_cluster_list = plot.load_cluster_dictionary(args.output_folder)
# Plot and save the Davies-Bouldin Index vs Number of Clusters
plot.plot_davies_bouldin(loaded_cluster_list, args.output_folder)

<p style="font-size:19px; text-align:left; font-weight:bold">4) Plot results.

Specify the parameters to plot the results and create figures. The Python script "plot_som_results.py" creates .png files of the results in som space, geospace and also creates boxplots.

In [ ]:
import configs.argsPlot
import src.plot_som_results as plot

argsP = configs.argsPlot.Args()

argsP.outsomfile= args.output_file_somspace   # som calculation somspace output text file
argsP.som_x= args.som_x         # som x dimension
argsP.som_y= args.som_y         # som y dimension
argsP.input_file= args.input_file   # Input file(*.lrn)
argsP.dir= args.output_folder            # Input file(*.lrn) or directory where som.dictionary was safet to (/output/som.dictionary)
argsP.grid_type= 'rectangular' # grid type (square or hexa), (rectangular or hexagonal)
argsP.redraw='true'       # whether to draw all plots, or only those required for clustering (true: draw all. false:draw only for clustering).
argsP.outgeofile=args.output_file_geospace     # som geospace results txt file
argsP.dataType='grid'       # Data type (scatter or grid)
argsP.noDataValue='-9999'    # noData value

plot.run_plotting_script(argsP)

Move figures into a sub folder, since "plot_som_results.py" does not override existing files. If the destination folder does not exist, it is created here. All file names are stored in a list that is used in the next step to show all output figures.

In [ ]:
import shutil
import os
import glob

file_path = args.output_folder
file_patterns = ["geoplot_*.png", "somplot_*.png", "boxplot_*.png", "db_score.png"]
destination_path = file_path+"/plots/"


# Lists to store matching files with their corresponding destination paths
all_figs = []
all_figs_lable = []

# Create the destination folder if it doesn't exist
if not os.path.exists(destination_path):
    os.makedirs(destination_path)
else:
    # Delete all files in the destination folder
    files_to_delete = glob.glob(os.path.join(destination_path, "*"))
    for file_to_delete in files_to_delete:
        os.remove(file_to_delete)

for file_pattern in file_patterns:
    # Use glob to get all files with the specified pattern
    matching_files = glob.glob(os.path.join(file_path, file_pattern))

    # Add matching files and their corresponding destination paths to the lists
    all_figs.extend([os.path.join(destination_path, os.path.basename(file)) for file in matching_files])
    all_figs_lable.extend([os.path.basename(file) for file in matching_files])

    # Move each matching file to the destination folder and overwrite existing files if necessary
    for source_file in matching_files:
        file_name = os.path.basename(source_file)
        destination_file_path = os.path.join(destination_path, file_name)

        # If the file already exists in the destination folder, delete it first
        if os.path.exists(destination_file_path):
            os.remove(destination_file_path)

        # Move the file to the destination folder
        shutil.move(source_file, destination_file_path)

print(all_figs)

Show all figures that were produced by "plot_som_results.py" and moved into a subfolder.

Boxplot:
> x axis is Cluster ID </br>
> Z axis is original data value </br>
> legend: number of data points in each cluster </br>

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
import ipyplot

# Clear Matplotlib cache
plt.close('all')

# Clear output
clear_output(wait=True)

images = all_figs
labels = all_figs_lable
tabs = [image.split('_')[-2] for image in labels]

print("List of figures:")
print(labels)
#print(tabs)

# Plot the images
#ipyplot.plot_images(images, max_images=50, img_width=250)
ipyplot.plot_class_representations(images,  labels, img_width=200, show_url=False)
ipyplot.plot_class_tabs(images, tabs, max_imgs_per_tab=50, img_width=400)